https://www.kaggle.com/c/word2vec-nlp-tutorial#part-2-word-vectors

In [23]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd       
import re
from bs4 import BeautifulSoup



In [37]:
train = pd.read_csv("../data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3, encoding='utf8')
test = pd.read_csv("../data/testData.tsv", header=0, delimiter="\t", quoting=3, encoding='utf8')

In [24]:
import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
#print stopwords.words("english") 

In [25]:
train.shape

(25000, 3)

In [26]:
train.columns.values

array([u'id', u'sentiment', u'review'], dtype=object)

In [27]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words) 

In [28]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [29]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

# print "Parsing sentences from unlabeled set"
# for review in unlabeled_train["review"]:
#     sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/selenb/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/selenb/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


NameError: name 'unlabeled_train' is not defined

In [31]:
print len(sentences)

266551


In [33]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-02-19 20:13:50,798 : INFO : 'pattern' package not found; tag filters are not available for English
2019-02-19 20:13:50,810 : INFO : collecting all words and their counts
2019-02-19 20:13:50,811 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-19 20:13:50,893 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-02-19 20:13:50,981 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-02-19 20:13:51,096 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-02-19 20:13:51,181 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-02-19 20:13:51,264 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-02-19 20:13:51,356 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-02-19 20:13:51,474 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-02-19 20:13:51,559 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-02-19 20:13:51,645 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-02-19 20:13:51,746 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-02-19 20:13:51,848 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

In [34]:
model.doesnt_match("man woman child kitchen".split())

/Users/selenb/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [35]:
#model.most_similar("man")

/Users/selenb/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'woman', 0.6567870378494263),
 (u'soldier', 0.6391888856887817),
 (u'boy', 0.6235976815223694),
 (u'doctor', 0.6222869157791138),
 (u'scientist', 0.5752479434013367),
 (u'lady', 0.5748927593231201),
 (u'cop', 0.5698310136795044),
 (u'guy', 0.5650266408920288),
 (u'person', 0.561811625957489),
 (u'monk', 0.5316123366355896)]

In [49]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000 == 0:
            print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [50]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


/Users/selenb/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
svm_model = SVC(kernel='linear', C=1E10)
svm_model.fit(trainDataVecs, train["sentiment"])

In [ ]:
# Test & extract results 
result = svm_model.predict( testDataVecs )